### User Authentication

In [1]:
import requests
from pyquery import PyQuery as pq

In [2]:
baseUrl="http://quotes.toscrape.com/"   #main URL
loginUrl="http://quotes.toscrape.com/login" #POST URL (Credentials and results)
logoutUrl="http://quotes.toscrape.com/logout"

In [3]:
user = "test" # <input type="text" class="form-control" id="username" name="username" />
csrf =''          # <input type="hidden" name="csrf_token" value=...... />
passw = "password" # <input type="password" class="form-control" id="password" name="password" />

#### Form data collection

In [4]:
html = requests.get(loginUrl)
response = pq(html.content)

In [5]:
html.headers

{'Date': 'Sat, 11 Mar 2023 12:42:55 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '1869', 'Connection': 'keep-alive', 'Vary': 'Cookie', 'Set-Cookie': 'session=eyJjc3JmX3Rva2VuIjoiZEhMV0FEcUlWUFJrb2hmWGNtZ01lbmpTaWJsckN0QlVaWWFLeEdOcEpUeUZ2d3pFdU9RcyJ9.ZAx3Tw.QmpJe6IoX-MtTUWCFS5B8fltWoo; HttpOnly; Path=/'}

In [6]:
html.headers['Set-Cookie']

'session=eyJjc3JmX3Rva2VuIjoiZEhMV0FEcUlWUFJrb2hmWGNtZ01lbmpTaWJsckN0QlVaWWFLeEdOcEpUeUZ2d3pFdU9RcyJ9.ZAx3Tw.QmpJe6IoX-MtTUWCFS5B8fltWoo; HttpOnly; Path=/'

In [7]:
html.cookies

<RequestsCookieJar[Cookie(version=0, name='session', value='eyJjc3JmX3Rva2VuIjoiZEhMV0FEcUlWUFJrb2hmWGNtZ01lbmpTaWJsckN0QlVaWWFLeEdOcEpUeUZ2d3pFdU9RcyJ9.ZAx3Tw.QmpJe6IoX-MtTUWCFS5B8fltWoo', port=None, port_specified=False, domain='quotes.toscrape.com', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False)]>

In [8]:
csrf = response.find('form:first input[name="csrf_token"]').attr('value') # <input type="hidden" name="csrf_token" value=...... />
username = response.find('input[id="username"]').attr('name')
password = response.find('input[id="password"]').attr('name')

In [9]:
[csrf,username,password]

['dHLWADqIVPRkohfXcmgMenjSiblrCtBUZYaKxGNpJTyFvwzEuOQs',
 'username',
 'password']

#### Create payload or data to be submitted as POST

##### Collect as much information as possible from DevTools on Headers, Payload format, Cookies and more

In [10]:
payload = {"csrf_token":csrf, username:user, password:passw}

# postHTML = requests.post(loginUrl,data=payload)            #Not Working
# postHTML = requests.post(loginUrl,data=payload,headers={}) #Not Working

postHTML = requests.post(loginUrl,data=payload,headers={'Cookie':html.headers['Set-Cookie']})

In [11]:
payload

{'csrf_token': 'dHLWADqIVPRkohfXcmgMenjSiblrCtBUZYaKxGNpJTyFvwzEuOQs',
 'username': 'test',
 'password': 'password'}

#### Verify if the URL has been rediected

In [12]:
postHTML.url

'http://quotes.toscrape.com/'

#### check if logout link exists

In [13]:
postHTML.content[:650] 

b'<!DOCTYPE html>\n<html lang="en">\n<head>\n\t<meta charset="UTF-8">\n\t<title>Quotes to Scrape</title>\n    <link rel="stylesheet" href="/static/bootstrap.min.css">\n    <link rel="stylesheet" href="/static/main.css">\n</head>\n<body>\n    <div class="container">\n        <div class="row header-box">\n            <div class="col-md-8">\n                <h1>\n                    <a href="/" style="text-decoration: none">Quotes to Scrape</a>\n                </h1>\n            </div>\n            <div class="col-md-4">\n                <p>\n                \n                    <a href="/logout">Logout</a>\n                \n                </p>\n            </div>\n  '

In [104]:
verify = pq(postHTML.content).find('a[href="/logout"]').text()
verify

'Logout'

#### Logout from the Page

In [85]:
quotePage = requests.get(logoutUrl)
quotePage.content

b'<!DOCTYPE html>\n<html lang="en">\n<head>\n\t<meta charset="UTF-8">\n\t<title>Quotes to Scrape</title>\n    <link rel="stylesheet" href="/static/bootstrap.min.css">\n    <link rel="stylesheet" href="/static/main.css">\n</head>\n<body>\n    <div class="container">\n        <div class="row header-box">\n            <div class="col-md-8">\n                <h1>\n                    <a href="/" style="text-decoration: none">Quotes to Scrape</a>\n                </h1>\n            </div>\n            <div class="col-md-4">\n                <p>\n                \n                    <a href="/login">Login</a>\n                \n                </p>\n            </div>\n        </div>\n    \n\n<div class="row">\n    <div class="col-md-8">\n\n    <div class="quote" itemscope itemtype="http://schema.org/CreativeWork">\n        <span class="text" itemprop="text">\xe2\x80\x9cThe world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.\xe2\x80\